In [31]:
import pandas as pd
import numpy as np
from sklearn.tree import _tree
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import export_text

# ============================================================================
# 1. CHARGEMENT ET ENCODAGE DES DONNÉES
# ============================================================================

In [4]:
df = pd.read_csv("../DATAS/ANSTAT2021_clusters_PC.csv")

In [ ]:
# Sauvegarde des données originales pour référence
df_original = df.copy()

In [ ]:
# Variables catégorielles à encoder
cat_vars = ['sex', 'marital_status', 'city', 'milieu_resid', 'region_name']

# Dictionnaires pour stocker les encoders et les mappings inversés
label_encoders = {}
inverse_mappings = {}  # Pour reconvertir code → label original

for col in cat_vars:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

    # Créer le mapping inverse : {code_numérique: label_original}
    inverse_mappings[col] = dict(enumerate(le.classes_))


print("✓ Encodage terminé")
print(f"Mappings inversés disponibles pour: {list(inverse_mappings.keys())}")
df

✓ Encodage terminé
Mappings inversés disponibles pour: ['sex', 'marital_status', 'city', 'milieu_resid', 'region_name']


,cluster,age_num,sex,marital_status,city,milieu_resid,region_name,bancarise
0,17,29,0,0,1,1,1,1
1,12,17,0,0,1,1,1,0
2,1,15,1,0,1,1,1,0
3,12,12,0,0,1,1,1,0
4,22,34,0,2,1,1,1,0
...,...,...,...,...,...,...,...,...
64469,89,11,1,0,395,0,23,0
64470,89,7,1,0,395,0,23,0
64471,89,10,1,0,395,0,23,0
64472,89,4,1,0,395,0,23,0


# ============================================================================
# 2. PRÉPARATION DES DONNÉES POUR L'ENTRAÎNEMENT
# ============================================================================

In [7]:
y = df['cluster']
X = df.drop(columns=['cluster'])

In [ ]:
features = X.columns.tolist()
features

In [9]:
# (Optionnel) split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [33]:
print(f"\n✓ Données préparées:")
print(f"  - Features: {features}")
print(f"  - Train: {len(X_train)}, Test: {len(X_test)}")


✓ Données préparées:
  - Features: ['age_num', 'sex', 'marital_status', 'city', 'milieu_resid', 'region_name', 'bancarise']
  - Train: 45131, Test: 19343


# ================================================================
# 3. Entraîner un arbre de décision pour "expliquer" les clusters
# ================================================================

In [12]:
tree_clf = DecisionTreeClassifier(
    max_depth=4,   # limite la profondeur pour garder des règles lisibles
    min_samples_leaf=30,  # évite des règles sur 2-3 individus
    random_state=42
)

In [13]:
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, min_samples_leaf=30, random_state=42)

In [14]:
print("Score (accuracy) de l'arbre sur le test :", tree_clf.score(X_test, y_test))

Score (accuracy) de l'arbre sur le test : 0.370263144289924


In [15]:
param_grid = {
    "max_depth": [3, 4, 5, 6, 8, None],
    "min_samples_leaf": [1, 5, 10, 20, 30],
    "criterion": ["gini", "entropy"]
}

dt = DecisionTreeClassifier(random_state=42)

In [16]:
grid = GridSearchCV(
    dt,
    param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)
grid.fit(X_train, y_train)

best_dt = grid.best_estimator_
print(grid.best_params_)

{'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1}


In [ ]:
y_pred = best_dt.predict(X_test)

print("Accuracy :", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))   # precision, recall, f1 par cluster

In [ ]:
print(confusion_matrix(y_test, y_pred))

# ============================================================================
# 4. EXTRACTION DES RÈGLES LISIBLES (VALEURS ORIGINALES)
# ============================================================================

In [48]:
def decode_threshold_to_labels(feature, threshold, operator):
    """
    Convertit un threshold numérique en labels originaux
    """
    if feature not in inverse_mappings:
        # Pour age_num ou bancarise (pas de décodage nécessaire)
        if feature == "age_num":
            return f"{int(threshold)} ans"
        return f"{threshold:.2f}"
    
    mapping = inverse_mappings[feature]
    
    # Récupérer tous les codes possibles
    all_codes = sorted(mapping.keys())
    
    if operator == "<=":
        # Codes <= threshold
        selected_codes = [c for c in all_codes if c <= threshold]
    else:  # ">"
        # Codes > threshold
        selected_codes = [c for c in all_codes if c > threshold]
    
    # Convertir en labels originaux
    labels = [mapping[c] for c in selected_codes]
    
    return labels

In [49]:
def format_condition(feature, operator, threshold):
    """
    Formate une condition en langage naturel avec valeurs originales
    Optimise l'affichage en utilisant 'PAS DANS' quand c'est plus court
    """
    decoded = decode_threshold_to_labels(feature, threshold, operator)
    
    if isinstance(decoded, list):
        # Nettoyer les types numpy
        decoded_clean = [str(v).replace('np.int64(', '').replace(')', '') for v in decoded]
        
        if len(decoded_clean) == 0:
            return f"{feature} (aucune valeur)"
        elif len(decoded_clean) == 1:
            return f"{feature} = '{decoded_clean[0]}'"
        else:
            # Calculer si "PAS DANS" serait plus court
            if feature in inverse_mappings:
                all_values = list(inverse_mappings[feature].values())
                all_values_clean = [str(v).replace('np.int64(', '').replace(')', '') for v in all_values]
                excluded = [v for v in all_values_clean if v not in decoded_clean]
                
                # Si moins de valeurs exclues et que ça représente < 50% du total
                if len(excluded) > 0 and len(excluded) < len(decoded_clean) and len(excluded) <= 10:
                    values_str = "', '".join(excluded)
                    return f"{feature} PAS DANS ['{values_str}']"
            
            # Sinon, afficher toutes les valeurs incluses
            values_str = "', '".join(decoded_clean)
            return f"{feature} ∈ ['{values_str}']"
    else:
        return f"{feature} {operator} {decoded}"

In [50]:
def extract_rules_with_original_values(tree_model, feature_names):
    """
    Extrait toutes les règles de l'arbre avec les valeurs originales
    """
    tree = tree_model.tree_
    
    def recurse(node_id=0, current_conditions=None):
        if current_conditions is None:
            current_conditions = []
        
        rules = []
        
        # Feuille : on a trouvé une règle complète
        if tree.feature[node_id] == _tree.TREE_UNDEFINED:
            cluster = int(np.argmax(tree.value[node_id][0]))
            n_samples = int(tree.n_node_samples[node_id])
            rules.append({
                "conditions": current_conditions.copy(),
                "cluster": cluster,
                "n_samples": n_samples
            })
            return rules
        
        # Nœud interne : on continue la récursion
        feature = feature_names[tree.feature[node_id]]
        threshold = tree.threshold[node_id]
        
        # Branche gauche (<=)
        left_conditions = current_conditions + [(feature, "<=", threshold)]
        rules.extend(recurse(tree.children_left[node_id], left_conditions))
        
        # Branche droite (>)
        right_conditions = current_conditions + [(feature, ">", threshold)]
        rules.extend(recurse(tree.children_right[node_id], right_conditions))
        
        return rules
    
    return recurse()

In [51]:
# Extraction des règles
print("\n⏳ Extraction des règles avec valeurs originales...")
rules_raw = extract_rules_with_original_values(best_dt, features)


⏳ Extraction des règles avec valeurs originales...


In [52]:
# Formatage lisible
rules_readable = []
for rule in rules_raw:
    conditions_text = []
    for feature, op, threshold in rule["conditions"]:
        cond_str = format_condition(feature, op, threshold)
        conditions_text.append(cond_str)
    
    rule_text = " ET ".join(conditions_text)
    
    rules_readable.append({
        "Règle_ID": len(rules_readable) + 1,
        "Conditions": rule_text,
        "Cluster_Assigné": rule["cluster"],
        "Nombre_Individus": rule["n_samples"]
    })

rules_df = pd.DataFrame(rules_readable)
print(f"✓ {len(rules_df)} règles extraites")
rules_df

✓ 804 règles extraites


,Règle_ID,Conditions,Cluster_Assigné,Nombre_Individus
0,1,marital_status = '0' ET milieu_resid = '0' ET ...,71,388
1,2,marital_status = '0' ET milieu_resid = '0' ET ...,80,11
2,3,marital_status = '0' ET milieu_resid = '0' ET ...,7,6
3,4,marital_status = '0' ET milieu_resid = '0' ET ...,27,189
4,5,marital_status = '0' ET milieu_resid = '0' ET ...,80,1
...,...,...,...,...
799,800,marital_status PAS DANS ['0'] ET bancarise > 0...,76,1
800,801,marital_status PAS DANS ['0'] ET bancarise > 0...,11,41
801,802,marital_status PAS DANS ['0'] ET bancarise > 0...,11,233
802,803,marital_status PAS DANS ['0'] ET bancarise > 0...,29,9


# ============================================================================
# 5. SAUVEGARDE DES RÈGLES ET DU MODÈLE
# ============================================================================

In [53]:
rules_df.to_csv("regles_segmentation_lisibles.csv", index=False, encoding="utf-8")
print("✓ Règles sauvegardées: regles_segmentation_lisibles.csv")

✓ Règles sauvegardées: regles_segmentation_lisibles.csv


In [54]:
# Sauvegarder le modèle et les encoders
with open("modele_clustering.pkl", "wb") as f:
    pickle.dump({
        "model": best_dt,
        "label_encoders": label_encoders,
        "inverse_mappings": inverse_mappings,
        "features": features
    }, f)
print("✓ Modèle sauvegardé: modele_clustering.pkl")

✓ Modèle sauvegardé: modele_clustering.pkl


In [55]:
# Afficher un échantillon de règles
print("\n" + "="*80)
print("EXEMPLE DE RÈGLES EXTRAITES (5 premières):")
print("="*80)
print(rules_df.head(10).to_string(index=False))


EXEMPLE DE RÈGLES EXTRAITES (5 premières):
 Règle_ID                                                                                                                                                                                                                                                                                            Conditions  Cluster_Assigné  Nombre_Individus
        1                                                                                                                                                                    marital_status = '0' ET milieu_resid = '0' ET bancarise <= 0.50 ET sex = '0' ET region_name ∈ ['0', '1', '2'] ET age_num <= 27 ans               71               388
        2                                                                                                                                                marital_status = '0' ET milieu_resid = '0' ET bancarise <= 0.50 ET sex = '0' ET region_name ∈ ['0', '1', '2'] ET age_num > 27

# ============================================================================
# 6. FONCTION D'ASSIGNATION POUR NOUVELLES DONNÉES
# ============================================================================

In [56]:
def assigner_nouveaux_individus(csv_path, model_path="modele_clustering.pkl"):
    """
    Assigne des clusters à de nouveaux individus
    
    Args:
        csv_path: Chemin vers le CSV avec les nouvelles données
        model_path: Chemin vers le modèle sauvegardé
    
    Returns:
        DataFrame avec les clusters assignés
    """
    # Charger le modèle et les encoders
    with open(model_path, "rb") as f:
        saved_data = pickle.load(f)
    
    model = saved_data["model"]
    label_encoders = saved_data["label_encoders"]
    features = saved_data["features"]
    
    # Charger les nouvelles données
    new_data = pd.read_csv(csv_path)
    print(f"✓ {len(new_data)} nouveaux individus chargés")
    
    # Encoder les variables catégorielles avec les MÊMES encoders
    new_data_encoded = new_data.copy()
    for col in label_encoders.keys():
        if col in new_data_encoded.columns:
            le = label_encoders[col]
            # Gérer les valeurs inconnues
            new_data_encoded[col] = new_data_encoded[col].apply(
                lambda x: le.transform([x])[0] if x in le.classes_ else -1
            )
    
    # Prédiction
    X_new = new_data_encoded[features]
    clusters = model.predict(X_new)
    
    # Ajouter les résultats
    result = new_data.copy()
    result['Cluster_Assigné'] = clusters
    
    return result


In [57]:
# Exemple d'utilisation commenté:
nouvelles_assignations = assigner_nouveaux_individus("../DATAS/ANSTAT2021_clusters_PC.csv")
nouvelles_assignations.to_csv("resultats_assignation.csv", index=False)

✓ 64474 nouveaux individus chargés


In [ ]:
print("\n" + "="*80)
print("✓ PROCESSUS TERMINÉ")
print("="*80)
print("\nFichiers générés:")
print("  1. regles_segmentation_lisibles.csv - Règles en langage naturel")
print("  2. modele_clustering.pkl - Modèle + encoders pour nouvelles données")
print("\nPour assigner de nouveaux individus:")
print("  nouvelles = assigner_nouveaux_individus('votre_fichier.csv')")